<a href="https://colab.research.google.com/github/areyesan/Keras_tools/blob/main/keras_tuner_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf



In [ ]:
pip install -q -U keras-tuner


     |████████████████████████████████| 135 kB 6.9 MB/s 
     |████████████████████████████████| 1.6 MB 57.9 MB/s 


In [ ]:
(X_train, y_train), (X_test, y_test)= tf.keras.datasets.mnist.load_data()


11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
(X_train.shape, y_train.shape), (X_test.shape, y_test.shape)


(((60000, 28, 28), (60000,)), ((10000, 28, 28), (10000,)))

In [ ]:
X_train=X_train/255.0
X_test=X_test/255.0


In [ ]:
from tensorflow.python.ops import losses
def model_builder(hp):
  model=tf.keras.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape=(28,28)))

  hp_activation=hp.Choice('activation', values=['relu', 'tanh'])
  hp_layer_1=hp.Int('layer_1', min_value=1, max_value=1000, step=100)
  hp_layer_2=hp.Int('layer_2', min_value=1, max_value=1000, step=100)
  hp_lr=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.add(tf.keras.layers.Dense(units=hp_layer_1, activation=hp_activation))
  model.add(tf.keras.layers.Dense(units=hp_layer_2, activation=hp_activation))
  model.add(tf.keras.layers.Dense(10, activation='softmax'))

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_lr),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])
  return model


In [ ]:
import keras_tuner as kt   

In [ ]:
tuner=kt.Hyperband(model_builder, 
                   objective='val_accuracy', 
                   max_epochs=10,
                   factor=3,
                   directory='dir',
                   project_name='x')

In [ ]:
stop_early=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

Trial 30 Complete [00h 00m 30s]
val_accuracy: 0.9637500047683716

Best val_accuracy So Far: 0.9770833253860474
Total elapsed time: 00h 10m 19s


In [ ]:
best_hps= tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
model = tuner.hypermodel.build(best_hps)

In [ ]:
history=model.fit(X_train, y_train, epochs=50,
                  validation_split=0.2,
                  callbacks=[stop_early])

Epoch 1/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2012 - accuracy: 0.9389 - val_loss: 0.1094 - val_accuracy: 0.9674
Epoch 2/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0841 - accuracy: 0.9735 - val_loss: 0.0951 - val_accuracy: 0.9723
Epoch 3/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0594 - accuracy: 0.9810 - val_loss: 0.0823 - val_accuracy: 0.9780
Epoch 4/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0459 - accuracy: 0.9849 - val_loss: 0.1014 - val_accuracy: 0.9722
Epoch 5/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0381 - accuracy: 0.9880 - val_loss: 0.1072 - val_accuracy: 0.9716
Epoch 6/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0286 - accuracy: 0.9906 - val_loss: 0.1163 - val_accuracy: 0.9737


In [ ]:
history.history 

{'loss': [0.20118969678878784,
  0.08414197713136673,
  0.05944468080997467,
  0.045886144042015076,
  0.03806003928184509,
  0.028617877513170242],
 'accuracy': [0.9388750195503235,
  0.973479151725769,
  0.981041669845581,
  0.9849374890327454,
  0.9880208373069763,
  0.9905833601951599],
 'val_loss': [0.10935055464506149,
  0.09512903541326523,
  0.08232449740171432,
  0.10139482468366623,
  0.107169970870018,
  0.1162736639380455],
 'val_accuracy': [0.9674166440963745,
  0.9723333120346069,
  0.9779999852180481,
  0.9721666574478149,
  0.9715833067893982,
  0.9736666679382324]}

In [ ]:
import pandas as pd

pd.DataFrame(history.history)

,loss,accuracy,val_loss,val_accuracy
0,0.201190,0.938875,0.109351,0.967417
1,0.084142,0.973479,0.095129,0.972333
2,0.059445,0.981042,0.082324,0.978000
3,0.045886,0.984937,0.101395,0.972167
4,0.038060,0.988021,0.107170,0.971583
5,0.028618,0.990583,0.116274,0.973667
